In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-a-strong-stockfish-wdl-model/sample_submission.csv
/kaggle/input/train-a-strong-stockfish-wdl-model/shuffled_fens.csv
/kaggle/input/train-a-strong-stockfish-wdl-model/test.csv


In [9]:
df = pd.read_csv('/kaggle/input/train-a-strong-stockfish-wdl-model/shuffled_fens.csv')
df.columns

Index(['FEN', 'WDL'], dtype='object')

In [10]:
df.sample(10)
df.head()
df['WDL'].unique()

array([1. , 0. , 0.5])

In [11]:
!pip install python-chess

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 48.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=2ce45482dbbac44b5079c9a18a4c444a54a6324637a93f25089fedba76d785e5
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import chess
import chess.engine
import chess.pgn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from joblib import Parallel, delayed

In [15]:
print(chess.Board('1r6/R3Pk2/6p1/2KB3p/5P2/8/8/8 b - - 5 50'))

. r . . . . . .
R . . . P k . .
. . . . . . p .
. . K B . . . p
. . . . . P . .
. . . . . . . .
. . . . . . . .
. . . . . . . .


In [16]:
def parse_fen(fen):
    board = chess.Board(fen)  # Extract the full FEN
    rows = board.board_fen().split("/")                               
    features = {
        'FEN' : board.board_fen(),
        'num_white_pawns': len(board.pieces(chess.PAWN, chess.WHITE)),
        'num_black_pawns': len(board.pieces(chess.PAWN, chess.BLACK)),
        'num_white_knights': len(board.pieces(chess.KNIGHT, chess.WHITE)),
        'num_black_knights': len(board.pieces(chess.KNIGHT, chess.BLACK)),
        'num_white_bishops': len(board.pieces(chess.BISHOP, chess.WHITE)),
        'num_black_bishops': len(board.pieces(chess.BISHOP, chess.BLACK)),
        'num_white_rooks': len(board.pieces(chess.ROOK, chess.WHITE)),
        'num_black_rooks': len(board.pieces(chess.ROOK, chess.BLACK)),
        'num_white_queens': len(board.pieces(chess.QUEEN, chess.WHITE)),
        'num_black_queens': len(board.pieces(chess.QUEEN, chess.BLACK)),
        #Castling options
        'can_white_cstl_ks' : int(board.has_kingside_castling_rights(chess.WHITE)),
        'can_white_cstl_qs' : int(board.has_queenside_castling_rights(chess.WHITE)),
        'can_black_cstl_ks' : int(board.has_kingside_castling_rights(chess.BLACK)),
        'can_black_cstl_qs' : int(board.has_queenside_castling_rights(chess.BLACK)),
        # Total pieces for each color
        'num_white_pieces': sum(len(board.pieces(piece_type, chess.WHITE)) for piece_type in chess.PIECE_TYPES),
        'num_black_pieces': sum(len(board.pieces(piece_type, chess.BLACK)) for piece_type in chess.PIECE_TYPES),
        # Board status. Each digit represents a status issue. 0 means no issues
        'status' : board.status(),
        # Number of BW turns
        'round' : board.fullmove_number,
        # Next turn. 1-white, 0-black
        'is_white_to_move': int(board.turn),
        "row1" : rows[0],
        "row2" : rows[1],
        "row3" : rows[2],
        "row4" : rows[3],
        "row5" : rows[4],
        "row6" : rows[5],
        "row7" : rows[6],
        "row8" : rows[7],
        
    }
    return features

In [ ]:
# Parallelize FEN parsing
def parse_fen_parallel(fen_list, n_jobs=-1):
    return Parallel(n_jobs=n_jobs)(delayed(parse_fen)(fen) for fen in fen_list)

# Apply parallel parsing
# Subset shuffle file
# fen_list = df[0:1000000]['FEN'].tolist()
fen_list = df[0:100000]['FEN']
fen_features = parse_fen_parallel(fen_list)
features_df = pd.DataFrame(fen_features)

features_df.head()

In [ ]:
#better to not select by index in case df changes.  Fix if time.
X = features_df.iloc[:, 1:19]
# y = df[0:100000]['WDL'].astype(int)

In [ ]:
# map WDL values to categories
y = pd.DataFrame(df[0:100000]['WDL'])

target_encoding = {1:1, 0:0, .5:2}
y['target'] = y['WDL'].map(target_encoding)

y = pd.DataFrame(y['target'])

print(y.sample(10))

In [2]:
from transformers import pipeline

ask_llm = pipeline(
    model = "Qwen/Qwen2.5-3B-Instruct"
)

print(ask_llm("1r6/R3Pk2/6p1/2KB3p/5P2/8/8/8"))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


[{'generated_text': "1r6/R3Pk2/6p1/2KB3p/5P2/8/8/8 w - - 0 1',\n    'e2e4 e7e5 g1f3 c7c5 e2e4 e7e5 g1f3 c7c5 d2d4 b8c6 d7d5 f2f4 d7d5 f2f4 c6b7 b8c6 d7d5 f2f4 c6b7 c5a6 b7c6 d7d5 f2f4 c6b7 b8c6 d7d5 f2f4 c6b7 b8c6 d7d5 f2f4 c6b7 b8c6 d7d5 f2f4 c6b7 b8c6 d7d5 f2f4 c6b7 b8c6 d7d5 f2f4 c6b7 b8c6 d7d5 f2f4 c6b7 b8c6 d7d5 f2f4 c6b7 b8c6 d7d5 f2f4 c6b7 b8c6 d7d5 f2f4 c6b7 b8c6 d7"}]


In [49]:

target_encoding = {1:'White wins', 0:'Black Wins', .5:'Draw'}

label_df = pd.DataFrame(df[0:100000]['WDL'])

label_df['target'] = label_df['WDL'].map(target_encoding)

split_df = pd.DataFrame(df[0:100000]['FEN'])
split_df['positions'] = split_df['FEN'].str.split(' ', expand=True)[0]
# sample_df = pd.concat([split_df['positions'], label_df], axis=1)
sample_df['sample'] = split_df['positions'] + "/n" + label_df['target']
sample_df = pd.DataFrame(sample_df['sample'])

sample_df.head()

,sample
0,r1b4r/1p3kb1/p2pp1p1/3q3p/3N1Pp1/2P3R1/PP1Q1BP...
1,r1bqkb1r/pp3ppp/2np1n2/8/2BNP3/8/PP3PPP/RNBQK2...
2,8/R7/5k2/2pqp3/6n1/8/8/1K6/nBlack Wins
3,1r6/R3Pk2/6p1/2KB3p/5P2/8/8/8/nWhite wins
4,2RB2bk/p2n3p/4Np2/8/2B3P1/1P3p1P/1P3P1K/2br4/n...


In [ ]:
from transformers import AutoTokenizer

def preprocess(sample):
    tokenizer = AutoTokenizer.from_pretrained(
        "Qwen/Qwen2.5-3B-Instruct"
    )
    
    sample = "1r6/R3Pk2/6p1/2KB3p/5P2/8/8/8	" + "\n" + "2"
    
    tokenized = tokenizer(sample,
             max_length=50,
             truncation=True,
              padding="max_length"
            )
    
    tokenized["labels"] = tokenized["input_ids"].copy()
    
    return tokenized 

sample_list = sample_df['sample']

def map_tokens_parallel(sample_list, n_jobs=-1):
    return Parallel(n_jobs=n_jobs)(delayed(preprocess)(sample) for sample in sample_list)

data = sample_list.map(preprocess)

In [55]:
# Want to try preprocess in parallel
# sample_list = sample_df['sample']

# def map_tokens_parallel(sample_list, n_jobs=-1):
#     return Parallel(n_jobs=n_jobs)(delayed(preprocess)(sample_list) for sample in sample_list)

# data = map_tokens_parallel(sample_list)

NameError: name 'preprocess' is not defined

In [ ]:
print(data["train"][9])

In [ ]:
       from peft import LoraConfig, get_peft_model, TaskType
       from transformers import AutoModelForCausalLM
       Import torch

       model = AutoModelForCausalLM.from_pretrained(
           "Qwen/Qwen2.5-3B-Instruct",
           torch_dtype=torch.float16
       )

       lora_config = LoraConfig(
           task_type=TaskType.CAUSAL_LM,
           target_modules=["q_proj", "k_proj", "v_proj"]
       )

       model = get_peft_model(model, lora_config)

In [ ]:
from transformers import TrainingArguments, Trainer

train_args = TrainingArguments(
    num_train_epochs = 2,
    learning_rate = 0.001,
    logging_steps = 25,
    fp16 = True
)

trainer = Trainer(
    args=train_args,
    model=model,
    train_dataset=data["train"]
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_mode("/kaggle/working/")
tokenizer.save_pretrained("/kaggle/working/")
# trainer.save_mode("/kaggle/temp/")
# tokenizer.save_pretrained("/kaggle/temp/")

In [ ]:
ask_trained_llm = pipeline(
    model = "/kaggle/working/",
    tokenizer = "/kaggle/working/"
)

print(ask_trained_llm("")[0]["generated_text"])